In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
num_cells = 800

In [3]:
freqs = [0 for i in range(num_cells)]

initial = 1250
final = 20000

oct_step = 0.005
step_factor = 2**oct_step

for i in range(num_cells):
    if i == 0:
        freqs[i] = initial
        continue

    freqs[i] = freqs[i-1]*step_factor
    
temp = 5


In [11]:
start_freq = 1250
end_freq = 20000

octave_step = 0.005
step_factor = 2**octave_step

frequencies = [start_freq]
current_freq = start_freq

while current_freq < end_freq:
    current_freq = current_freq*step_factor
    frequencies.append(current_freq)

temp = 5

In [4]:
conn_params = {'BW': {'AN_W': 2,
                      'AN_I2': 0.4,
                      'AN_P': 0.4,
                      'W_I2': 0.1,
                      'W_P': 0.1,
                      'I2_P': 0.6},
               'N': {'AN_W': 140,
                     'AN_I2': 48,
                     'AN_P': 48,
                     'W_I2': 15,
                     'W_P': 15,
                     'I2_P': 21},
               'strength': {'AN_W': 0.06,
                            'AN_I2': 0.55,
                            'AN_P': 0.25,
                            'W_I2': 1.4,
                            'W_P': 0.1,
                            'I2_P': 2.25}}

df = pd.DataFrame.from_dict(conn_params)


In [5]:
df

,BW,N,strength
AN_W,2.0,140,0.06
AN_I2,0.4,48,0.55
AN_P,0.4,48,0.25
W_I2,0.1,15,1.40
W_P,0.1,15,0.10
I2_P,0.6,21,2.25


In [8]:
cell_ids = {'AN': [i for i in range(num_cells)],
            'W': [i for i in range(num_cells)],
            'I2': [i for i in range(num_cells)],
            'P': [i for i in range(num_cells)]}

conns = {'AN_W': [],
         'AN_I2': [],
         'AN_P': [],
         'W_I2': [],
         'W_P': [],
         'I2_P': []}

freqs_arr = np.array(freqs)
freqs_round = np.round(freqs_arr,0)

for conn, row in df.iterrows():
    temp = 5

    source = conn.split('_')[0]
    source_ids = cell_ids[source]

    target = conn.split('_')[1]
    target_ids = cell_ids[target]

    conn_list = []

    for target_id in target_ids:

        target_freq = freqs_round[target_id]

        bw = row.BW
        bw_split = bw / 2
        num_source = int(row.N)

        lb_freq = int(target_freq * 2**-bw_split)
        if lb_freq < 1250: lb_freq = 1250
        ub_freq = int(target_freq * 2**bw_split)

        lb = (np.abs(freqs_round - lb_freq)).argmin()
        ub = (np.abs(freqs_round - ub_freq)).argmin()

        source_pool = source_ids[lb:ub]

        if target_id == 0:
            bw_whole = len(source_pool)*2
        if num_source > len(source_pool):
            num_source *= (len(source_pool)/bw_whole)

        try:
            source_rand = random.sample(source_pool,int(num_source))
        except ValueError:
            print(conn,target_id)

        conn_list.extend([[i,target_id] for i in source_rand])

    conns[conn] = conn_list


In [9]:
temp = 5